# Internet of Things

El microcontrolador que estamos usando es NodeMCU v1.0. ESP-12E.

Al encender el microcontrolador se carga el boot.py y luego el main.py.

Casi todos los main.py van a trabajar con un bucle while.

## NUNCA CONECTAR POSITIVO Y NEGATIVO CON UN CABLE!!!!!!!!

## Hello world:

In [ ]:
import machine
import utime

led = machine.Pin(2, machine.Pin.OUT)

while True:
    led.on()
    utime.sleep(1)
    led.off()
    utime.sleep(1)

## Estado del botón:

In [ ]:
import machine
import utime
led = machine.Pin(2, machine.Pin.OUT)
boton = machine.Pin(0, machine.Pin.IN)
while True:

    estado_boton = boton.value()
    print(estado_boton)
    led.value(estado_boton)
    utime.sleep(0.1)

## Pull.UP para definir voltaje a 0 y 1

In [ ]:
from machine import Pin
import utime

boton2 = Pin(16, Pin.PULL_UP)

while True:
    valor = boton2.value()
    print('valor: {}'.format(valor))
    utime.sleep(0.1)

# Hay que conectar un cable dupont al pin 16 y a tierra hace que el value sea 0

## Recreación salida analógica con una salida digital

Usamos PWM para dar una salida analógica mediante una digital. Nos permites más estados que 0 y 1.

In [ ]:
from machine import Pin, PWM
import utime

led = Pin(2, Pin.OUT)
led_pwm = PWM(led) # la frequencia se puede especificar ahí como segundo elemento de la tupla o por separado

led_pwm.freq(500)

while True:
    for i in range(1023):
        led_pwm.duty(i)
        utime.sleep_ms(3)
    for i in range(1023, 0, -1):
        led_pwm.duty(i)
        utime.sleep_ms(3)

Con PWM podemos usar servos y ccntrolar así el movimiento que hace, siendo `0` y `1` los valores límite y los intermedios, posiciones intermedias.

Podemos hacer lo mismo pero con un potenciómetro (que no tenemos) y depende del voltaje, da más intensidad o menos.

In [ ]:
from machine import Pin, PWM, ADC
import utime

led = Pin(2, Pin.OUT)
led_pwm = PWM(led)
led_pwm.freq(1000)

pot = ADC(0)

while True:
    lectura = pot.read()
    led_pwm.duty(lectura)
    print(lectura)
    utime.sleep(0.5)

## Los Timers:

Se usan con `machine.Timer()` y se usan para tareas recurrentes o para hacer contadores.

Con `Timer(-1)` tenemos un timer simulado por software y con otros números los de hardware.

Iniciamos con .init() y los parámetros son:
- `period`: El periodo, cada cuánto se activa
- `mode`: Tenemos el `ONE_SHOT`, que se activa una vez y el `PERIODIC`, que se activa cada los milisegundos indicados.
- `callback`: Podemos especificar un callback, usando funciones (incluyendo lambdas).

In [ ]:
from machine import Pin, PWM, ADC
import utime

tim = Timer(-1)

def texto(x): # x es el lugar en memoria de la instancia
    print('hola') # nunca poner un sleep aquí, ya que se romperá con el period de abajo

tim.init(period=1000, mode=Timer.PERIODIC, callback=texto)

Podemos usar `time.deinit()` para parar el timer. Si hacemos un sleep antes, el timer se va a estar ejecutando durante ese tiempo, lo que nos permite controlar las veces que se ejecuta.

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

tim = Timer(-1)

def texto(x):
    print('hola')

tim.init(period=1000, mode=Timer.PERIODIC, callback=texto)

utime.sleep(3)

tim.deinit()

La técnica usada anteriormente para leer el botón se llama pulling. Lee el estado periódicamente del botón. No funciona bien con sleeps muy altos.

Podemos usar un timer para lanzar un callback cuando se pulse. Para ello se usa el método `.irq()`, que tiene dos parámetros:
- `función`: La función que debe ejecutar
- `trigger`: Define lo que hará que se lance la señal.
    - `IRQ_FALLING`: Pulsación del botón
    - `IRQ_RISING`: Soltar el botón

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

def boton_callback(x):
    print('botón pulsado')

boton = Pin(0, Pin.IN)
boton.irq(boton_callback, trigger=Pin.IRQ_FALLING) # tráfico descendente

while True:
    estado = boton.value()
    print('x')
    utime.sleep(0.1)

Otra implementación:

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

boton_pulsado= False

def boton_callback(x):
    global boton_pulsado
    boton_pulsado = True

boton = Pin(0, Pin.IN)
boton.irq(boton_callback, trigger=Pin.IRQ_FALLING) # tráfico descendente

while True:
    print('x')
    if boton_pulsado:
        print("boton_pulsado")
        boton_pulsado = False
    utime.sleep(0.1)

En el mundo real hay un montón de ruido al pulsar el botón y no es tan fácil capturar el raising y el falling. Se puede lidiar con ese problema de forma manual o usando librerías específicas. Usando el código anterior, nos metemos en el bucle:

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

boton_pulsado = False
tiempo_inicio_boton = 0

def boton_callback(x):
    global boton_pulsado, tiempo_inicio_boton
    boton_pulsado = True
    tiempo_inicio_boton = utime.ticks_ms()    


boton = Pin(0, Pin.IN)
boton.irq(boton_callback, trigger=Pin.IRQ_FALLING)

while True:
    print("x")
    if boton_pulsado:
        print("boton pulsado marca {}".format(tiempo_inicio_boton))
        ahora =  utime.ticks_ms()  
        tiempo_boton = utime.ticks_diff(ahora, tiempo_inicio_boton)

        print(tiempo_boton)
        if tiempo_boton < 50:
            utime.sleep_ms(50-tiempo_boton)
        estado = boton.value()
        if estado:
            print("esto es ruido, no hacer nada")
        else:
            print("boton pulsado de verdad")
        utime.sleep(0.5) #antirrebotes
        boton_pulsado = False
    utime.sleep(0.1)

Para más implementaciones: http://docs.micropython.org/en/v1.9.3/pyboard/pyboard/tutorial/debounce.html

### Otras librerías interesantes:
- `machine`
- `utime`
- `esp`

In [ ]:
from machine import Pin
from utime import sleep

class Boton:
    def __init__(self, pin_n):
        self.boton = Pin(pin_n, Pin.IN) # PULL_UP
        self.boton.irq(self.pulsado_cb, Pin.IRQ_FALLING)
        self.pulsado = False

    def pulsado_cb(self, inst):
        self.pulsado = True

    def get_pulsado(self):
        if not self.pulsado:
            return False
        self.pulsado = False
        return True



boton = Boton(0)


while True:
    if boton.get_pulsado():
        print('se ha pulsado el boton')
    print('x', end="")
    sleep(0.5)

Mismo código pero con las ayudas para detectar bien el botón

In [ ]:
from machine import Pin
from utime import sleep
import micropython

class Boton:
    def __init__(self, pin_n):
        self.boton = Pin(pin_n, Pin.IN) # PULL_UP
        self.boton.irq(self.pulsado_cb, Pin.IRQ_FALLING)
        self.pulsado = False

    def pulsado_cb(self, inst):
        micropython.schedule(self.post_pulsado_cb, 0)

    def post_pulsado_cb(self, inst):
        # aqui puedo hacer operaciones más caras, ya que lo hace el rtOS
        # self.pulsado = True
        self.boton.irq(self.liberado_cb, Pin.IRQ_RISING)

    def liberado_cb(self, inst):
        micropython.schedule(self.post_liberado_cb, 0)

    def post_liberado_cb(self, inst):
        self.pulsado = True
        self.boton.irq(self.pulsado_cb, Pin.IRQ_FALLING)

    def get_pulsado(self):
        if not self.pulsado:
            return False
        self.pulsado = False
        return True



boton = Boton(0)


while True:
    if boton.get_pulsado():
        print('\nse ha pulsado el boton')
    print('x', end="")
    sleep(0.5)


# Redes

In [ ]:
import network

red = network.WLAN(network.AP_IF)
red.active(True)
red.config(essid="MicroPython", password="12345678")

##  AP con web para controlar el led

In [ ]:
import network
import usocket as socket
from machine import Pin

def web_page(estado):
    html = """<html><head> <title>Ejemplo 1</title> <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="icon" href="data:;base64,iVBORw0KGgo="><style>html{font-family: Helvetica; display:inline-block; margin: 0px auto; text-align: center;}
    h1{color: #0F3376; padding: 2vh;}p{font-size: 1.5rem;}.button{display: inline-block; background-color: #e7bd3b; border: none;
    border-radius: 4px; color: white; padding: 16px 40px; text-decoration: none; font-size: 30px; margin: 2px; cursor: pointer;}
    .button2{background-color: #4286f4;}</style></head><body> <h1>CONTROL DE LED</h1>
    <p>Estado LED: <strong> """ + estado + """ </strong></p><p><a href="/?led=on"><button class="button">ON</button></a></p>
    <p><a href="/?led=off"><button class="button button2">OFF</button></a></p></body></html>"""
    return html

led = Pin(2, Pin.OUT)
led.value(1)

red = network.WLAN(network.AP_IF)
red.active(True)
red.config(essid="MicroPython", password="12345678")

print(red.ifconfig())

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(('0.0.0.0', 80))
sock.listen(5)
# sock.settimeout(30)

while True:
    print('Esperando conexion...')
    conn, addr = sock.accept()
    print(addr)
    request = conn.recv(1024)
    request_str = request.decode()
    print(request_str)
    respuesta = '<h1>Hola, mundo</h1> yo soy Rober'
    ledon = request_str.find("/?led=on")
    print(ledon)
    ledoff = request_str.find("/?led=off")
    print(ledoff)

    estado_led = led.value()
    estado_str = "ENCENDIDO" if not estado_led else "APAGADO"

    if  ledon == 4:
        led.value(0)
        respuesta = "Led encendido"
    elif ledoff == 4:
        respuesta = "Led apagado"
        led.value(1)

    conn.sendall('HTTP/1.1 200 OK\n')
    conn.sendall('Content-Type: text/html\n')
    conn.sendall('Connection: close\n\n')
    conn.sendall(web_page(estado_str))
    conn.close()


## Conexión a tu wifi

In [ ]:
from machine import Pin
import network
import utime as time

from credenciales import ssid, password # hay que crear el fichero credenciales


# Configurar hardware
led = Pin(2, Pin.OUT)


# Conexión wifi
led.value(0)
print('\nConectandose a wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected(): # Espera hasta que conecte
    time.sleep(0.1)

print('conectado!')
print(red.ifconfig())
led.value(1)

In [1]:
# Fichero credenciales

ssid = "wifi"
password = "contraseña"

## Misma web para encender o apagar led

In [ ]:
from machine import Pin
import network
import utime as time

from credenciales import ssid, password # hay que crear el fichero credenciales
from corneto import Corneto

# Configurar hardware
led = Pin(2, Pin.OUT)


# Conexión wifi
led.value(0)
print('\nConectandose a wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected(): # Espera hasta que conecte
    time.sleep(0.1)

print('conectado!')
print(red.ifconfig())
led.value(1)

web = Corneto()

def home(x):
    contexto = {
        "tiempo": str(time.ticks_ms() // 1000),

    }
    return("index.html", contexto)

def config(x):
    contexto = {}
    return("configuracion.html", contexto)

def led_encender(x):
    led.value(0)
    contexto = {}
    return("luz.html", contexto)

def led_apagar(x):
    led.value(1)
    contexto = {}
    return("luz.html", contexto)

web.add_view('/', home)
web.add_view('/configuracion', config)
web.add_view('/encender', led_encender)
web.add_view('/apagar', led_apagar)


web.run_server()

## Programa para enviar mensaje por UDP

In [ ]:
# Desde el microcontrolador

from machine import Pin
import network
import utime as time
import usocket as socket

from credenciales import ssid, password # hay que crear el fichero credenciales

# Configurar hardware
led = Pin(2, Pin.OUT)


# Conexión wifi
led.value(0)
print('\nConectandose 2a wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected(): # Espera hasta que conecte
    time.sleep(0.1)
print('conectado!')
print(red.ifconfig())
led.value(1)

while True:
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.settimeout(3)
    mensaje = b"Hola, mundo!"
    print(mensaje)
    sock.sendto(mensaje, ("192.168.1.255", 5005))
    # sock.setsockopt(SOL_SOCKET, SO_BROADCAST, 1) # esto en usocket no funciona bien
    try:
        data, addr = sock.recvfrom(512)
        print(addr)
        print(data)
    except OSError: # TIMEOUT
        print("no hay respuesta")
    sock.close()
    led.value(0)
    time.sleep(0.1)
    led.value(1)
    time.sleep(1)

In [2]:
# Desde servidor

import socket

UDP_PORT = 5005

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(("0.0.0.0", UDP_PORT))
print("Escuchando por el puerto {}".format(UDP_PORT))

try:
    while True:
        
        try:
            data, addr = sock.recvfrom(512)

        except socket.timeout:
            continue

        print(addr)
        print(data)
        sock.sendto(b'Te he oido', addr)

except KeyboardInterrupt:
    sock.close()

Escuchando por el puerto 5005
('192.168.1.115', 4097)
b'Hola, mundo!'
('192.168.1.115', 4097)
b'Hola, mundo!'
('192.168.1.115', 4097)
b'Hola, mundo!'
('192.168.1.115', 4097)
b'Hola, mundo!'
('192.168.1.115', 4097)
b'Hola, mundo!'
('192.168.1.115', 4097)
b'Hola, mundo!'


## Usar `ntptime` para obtener el datetime

In [ ]:
# el schedule nos da problemas

from machine import Pin, RTC
import network
import ntptime
from micropython import schedule
import utime as time
import usocket as socket

from credenciales import ssid, password # hay que crear el fichero credenciales

# Configurar hardware
led = Pin(2, Pin.OUT)


# Conexión wifi
led.value(0)
print('\nConectandose 2a wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected(): # Espera hasta que conecte
    time.sleep(0.1)
print('conectado!')
print(red.ifconfig())
led.value(1)

rtc = RTC()
rtc.datetime((2020,6,15,19,2,0,0,0)) # configuración de hora manual

ntptime.settime()
rtc.datetime()
print(rtc.datetime())

def texto_cb():
    print(rtc.datetime())

schedule.every(2).minutes.do(texto_cb)

while True:
    schedule.run_pending()
    time.sleep(1)

## Instalar urequests

In [ ]:
from machine import Pin
import utime as time
import network


from credenciales import ssid, password


led = Pin(2, Pin.OUT)


led.value(0)
print('\nConectandose a la wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected():
    time.sleep(0.1)
print('conectado!')
print(red.ifconfig())
led.value(1)

import upip
upip.install('micropython-urequests')

## Llamadas `API` usando JSON

In [ ]:
import ujson as json
from machine import Pin
import utime as time
import network
import urequests as requests

from credenciales import ssid, password

led = Pin(2, Pin.OUT)

led.value(0)
print('\nConectandose a la wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected():
    time.sleep(0.1)
print('conectado!')
print(red.ifconfig())
led.value(1)


codigo_ciudad = "773692"
url = "https://www.metaweather.com/api/location/{}/".format(codigo_ciudad)
r = requests.get(url)
if r.status_code is not 200:
    print("Error al acceder a metaweather, reseteando...")
    print('r.status')
    print('r.content')
    utime.sleep(10)
    reset()
print('Todo ok')
resultado = r.content.decode()

data = json.loads(resultado)

humedad = data["consolidated_weather"][1]['humidity']

print('La humedad mañana en Tenerife sera: {}'.format(humedad))

## Api para sacar el precio de las criptomonedas

In [ ]:
import ujson as json
from machine import Pin
import utime as time
import network
import urequests as requests

from credenciales import ssid, password

led = Pin(2, Pin.OUT)

led.value(0)
print('\nConectandose a la wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected():
    time.sleep(0.1)
print('conectado!')
print(red.ifconfig())
led.value(1)


def get_rate(currency):
    url = "https://api.coindesk.com/v1/bpi/currentprice/{}.json".format(currency)
    r = requests.get(url)
    if r.status_code is not 200:
        print("Error al acceder a coindesk, reseteando...")
        print('r.status')
        print('r.content')
        time.sleep(10)
        reset()
    resultado = r.content.decode()
    data = json.loads(resultado)
    rate = data["bpi"]["USD"]["rate"]
    return (rate)

crypto_code = "BTC"

while True:
    rate = get_rate(crypto_code)
    print('El precio del {} es {} USD'.format(crypto_code, rate))
    time.sleep(10)

## Encender y apagar leds usando el protocolo MQTT

In [ ]:
import ujson as json
from machine import Pin
import utime as time
import network
import urequests as requests
from umqtt.simple import MQTTClient
from credenciales import ssid, password

led = Pin(2, Pin.OUT)

led.value(0)
print('\nConectandose a la wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected():
    time.sleep(0.1)
print('conectado!')
print(red.ifconfig())
led.value(1)

def sub_cb(topic, msg):
    topic = topic.decode()
    msg = msg.decode()
    print("Me llegó por '{}' esto: '{}' ".format(topic, msg))
    if topic is "luz_mc_01":
        if msg == "on" :
            led.value(0)
            print('encendiendo led')
        elif msg == "off":
            led.value(1)
            print('apagando led')

id_cliente = "1k2hg3u1o2b3hkj1"
mqtt_server = "broker.hivemq.com"

topic = b"luz_mc_01"

client = MQTTClient(id_cliente, mqtt_server)
client.connect()
client.set_callback(sub_cb)
client.subscribe(topic)

# proximo_envio = time.ticks_ms() + 5000
#
# while True: # versión mejor
#     client.check_msg()
#     if time.ticks_ms() > proximo_envio:
#         client.publish(topic, b"on")
#         time.sleep(5)
#         client.publish(topic, b"off")
#         time.sleep(5)
#         proximo_envio = time.ticks_ms() + 5000
#     time.sleep_ms(100)


while True:
    client.check_msg()
    client.publish(topic, b"on")
    time.sleep(5)
    client.publish(topic, b"off")
    time.sleep(5)

## Enviar mensajes por MQTT pulsando el botón de la placa

In [ ]:
import ujson as json
from machine import Pin
import utime as time
import network
import urequests as requests
from umqtt.simple import MQTTClient
from credenciales import ssid, password

led = Pin(2, Pin.OUT)

led.value(0)
print('\nConectandose a la wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected():
    time.sleep(0.1)
print('conectado!')
print(red.ifconfig())
led.value(1)

boton_pulsado= False
def boton_callback(x):
    global boton_pulsado
    boton_pulsado = True

boton = Pin(0, Pin.IN)
boton.irq(boton_callback, trigger=Pin.IRQ_FALLING)

def sub_cb(topic, msg):
    topic = topic.decode()
    msg = msg.decode()

topic_pub = b"clase_eoi"

id_cliente = "1k2hg3u1o2b3hkj1"
mqtt_server = "broker.hivemq.com"

client = MQTTClient(id_cliente, mqtt_server)
client.connect()
client.set_callback(sub_cb)

while True:
    if boton_pulsado:
        print("boton_pulsado")
        client.publish(topic_pub, b"Soy Roberto pulsando el botón")
        boton_pulsado = False
    time.sleep(0.1)

## Juego del botón básico

### main.py

In [ ]:
from credenciales import ssid, password
import network
import utime as time

from juego import Juego

def conectar_wifi():
    print('\nConectandose a la wifi...', end='')
    red = network.WLAN(network.STA_IF)
    red.active(True)
    red.connect(ssid, password)
    while not red.isconnected():
        time.sleep(0.1)
    print('conectado!')
    print(red.ifconfig())

conectar_wifi()

juego = Juego()

juego.start()

### juego.py

In [ ]:
rom machine import Pin
import urandom as random
import utime as time

class Juego:
    def __init__(self):
        self.led = Pin(2, Pin.OUT)
        self.boton = Pin(0, Pin.IN)
        self.led.value(1)
        self.inicio = 0
        self.final = 0
        self.tiempo = 0

    def start(self):
        while True:
            print()
            print('Pulsa el boton cuando se encienda el led')
            self.encender_led_aleatorio()
            self.apagar_led_boton()
            self.tiempo = time.ticks_diff(self.final, self.inicio)
            print('Has apagado el botón en {} ms'.format(self.tiempo))

    def encender_led_aleatorio(self):
        aleatorio = random.getrandbits(12)
        aleatorio += 3000
        time.sleep_ms(aleatorio)
        self.led.value(0)
        self.inicio = time.ticks_ms()

    def apagar_led_boton(self):
        while self.boton.value() == 1:
            time.sleep_ms(1)
        self.led.value(1)
        self.final = time.ticks_ms()

## Juego del botón enviando por MQTT

### main.py

In [ ]:
from credenciales import ssid, password
import network
import utime as time

from juego import Juego

def conectar_wifi():
    print('\nConectandose a la wifi...', end='')
    red = network.WLAN(network.STA_IF)
    red.active(True)
    red.connect(ssid, password)
    while not red.isconnected():
        time.sleep(0.1)
    print('conectado!')
    print(red.ifconfig())

conectar_wifi()

juego = Juego("Rober .H")

juego.start()

### juego.py

In [ ]:
from machine import Pin
import urandom as random
import utime as time

from mimqtt import Mimqtt
from credenciales import ssid, password

class Juego:
    def __init__(self, jugador):
        self.led = Pin(2, Pin.OUT)
        self.boton = Pin(0, Pin.IN)
        self.led.value(1)
        self.jugador = jugador
        self.mimqtt = Mimqtt()


    def start(self):
        while True:
            print()
            print('Pulsa el boton cuando se encienda el led')
            self.encender_led_aleatorio()
            inicio = time.ticks_ms()
            self.apagar_led_boton()
            final = time.ticks_ms()
            tiempo = time.ticks_diff(final, inicio)
            if tiempo > 5000:
                print('Has tardado más de 5 segundos')
                print("\n")
                print('Reiniciando ...')
            if tiempo < 5000:
                print('Has apagado el botón en {} ms'.format(tiempo))
                self.mimqtt.enviar_mqtt(self.jugador, tiempo)

    def encender_led_aleatorio(self):
        aleatorio = random.getrandbits(12)
        aleatorio += 3000
        time.sleep_ms(aleatorio)
        self.led.value(0)


    def apagar_led_boton(self):
        while self.boton.value() == 1:
            time.sleep_ms(1)
        self.led.value(1)

### mimqtt.py

In [ ]:
import ujson as json

from umqtt.simple import MQTTClient


class Mimqtt():

    def enviar_mqtt(self, nombre, tiempo):
        id_cliente = "12hi3b1i2n98dno2jnd921"
        mqtt_server = "broker.hivemq.com"
        topic = b'curso_eoi'
        datos = {
            "Nombre": nombre,
            "Tiempo": tiempo
        }
        datos = json.dumps(datos)
        client = MQTTClient(id_cliente, mqtt_server)
        client.connect()
        client.publish(topic, datos)

# Sensor de proximidad

In [3]:
from machine import Pin, I2C
i2c = I2C(sda=Pin(5), scl=Pin(4))
i2c.scan() #comprobamos que es 57

ADDR_SENSOR = 0x39

i2c.writeto(ADDR_SENSOR, bytearray([0x12|0xA0])) # LE AVISAMOS QUE VAMOS QUEREMOS LEER
resultado = i2c.readfrom(ADDR_SENSOR, 1)[0] # LE INDICAMOS QUE QUEREMOS LEER UN 1
print(resultado)

NameError: name 'I2C' is not defined

## Sensor funcionando

### main.py

In [ ]:
from machine import Pin, I2C
import utime


i2c=I2C(sda=Pin(4), scl=Pin(5))  # instanciamos y configuramos bus I2C en los pines sda y scl
dispositivos_conectado = i2c.scan()  # manda mensajes por el bus i2c a todas las direcciones para ver que dispositivos contestan
# devuelve un listado de dispositivos conectados
print(dispositivos_conectado)  # NOTA las direcciones las muestra en decimal, normalmente usaremos hexadecimal para trabajar con el i2c

from apds9930 import APDS9930
sensor = APDS9930(i2c)  # creamos una instancia del sensor y le pasamos el manejador del i2c
# el manejador del i2c lo creamos aqui porque si tenemos varios sensores en el bus, le pasamos el mismo manejador a todos

sensor.activar_proximidad()  # este metodo modifica un registro interno del APDS9930 para activar el sensor de proximidad

print("Acerca la mano al sensor para activarlo")
while True:
    proximidad = sensor.get_proximidad()
    if proximidad is not 0:
        print("Activado! lectura {}".format(proximidad))
        utime.sleep_ms(100)  # para que no sature la consola con prints

### apds9930.py

In [ ]:
import utime
from micropython import const


# NOTA esto no es una libreria completa, es un ejemplo parcial para ilustrar lo que es el protocolo I2C
# se puede encontrar una libreria mas completa y funcional aqui:
# https://github.com/micropython-Chinese-Community/mpy-lib/tree/master/sensor/APDS9930


class APDS9930:
    I2C_ADDRESS = const(0x39)  # (57 en decimal) es la direccion de este sensor que es fija y no se puede cambiar
    # utilizamos const para indicar que es un valor fijo que no va a cambiar nunca. Es mas eficiente en memoria
    def __init__(self, i2c):
        self.i2c = i2c
        # Todo esto es la inicializacion del sensor segun viene descrita en el datasheet pagina 15
        ATIME = 0xFF # 2.7 ms – minimum ALS integration time
        WTIME = 0xFF # 2.7 ms – minimum Wait time
        PTIME = 0xFF # 2.7 ms – minimum Prox integration time
        PPULSE = 1 # Minimum prox pulse count
        self._write_reg_data(0, 0)  # Disable and Powerdown
        self._write_reg_data (0x01, ATIME)
        self._write_reg_data (0x02, PTIME)
        self._write_reg_data (0x03, WTIME)
        self._write_reg_data (0x0E, PPULSE)
        PDRIVE = 0  # 100mA of LED Power
        PDIODE = 0x20  # CH1 Diode
        PGAIN = 0  # 1x Prox gain
        AGAIN = 0  # 1x ALS gain
        self._write_reg_data (0x0F, PDRIVE | PDIODE | PGAIN | AGAIN)
        WEN = 8  # Enable Wait
        PEN = 4  # Enable Prox
        AEN = 2  # Enable ALS
        PON = 1  # Enable Power On
        self._write_reg_data(0, WEN | PEN | AEN | PON)  # self._write_reg_data(0, 0x0F)
        utime.sleep_ms(12)  # Wait for 12 ms
        CH0_data = self._read_word(0x14)
        CH1_data = self._read_word(0x16)
        Prox_data = self._read_word(0x18)
        # print(CH0_data)
        # print(CH1_data)
        # print(Prox_data)


    def _read_word(self, reg):
        """ Lee un word (2 bytes, 16 bits) de un registro del sensor """
        # es una implementacion del codigo de ejemplo que aparece en el datasheet pag 15
        registro_enmascarado = reg|0xA0  # para entender esta mascara mirar Command Register pag 19 datasheet
        # al writeto le pasamos la direccion y los bytes que queremos escribir
        # puede ser en formato b'\x11\x43\xa8' o con bytearray donde le pasamos una lista o tupla con los bytes
        # aunque sea un solo byte se lo tenemos que pasar asi, como lista o tupla (tupla mas eficiente)
        self.i2c.writeto(APDS9930.I2C_ADDRESS, bytearray((registro_enmascarado, )))
        # despues de decirle que registro queremos leer, lo leemos. En este caso como son dos bytes le pasamos
        # un 2 y nos devolvera un bytearray de 2 elementos con la lectura de ese registro y el siguiente
        r = self.i2c.readfrom(APDS9930.I2C_ADDRESS, 2)
        return r[0] + r[1]*256  # convertimos esos dos bytes en un entero (el primero el menos signigicativo)


    def _read_byte(self, reg):
        """ Lee un byte de un registro del sensor """
        # mirar comentarios de _read_word
        self.i2c.writeto(APDS9930.I2C_ADDRESS, bytearray([reg|0xA0]))
        t = self.i2c.readfrom(APDS9930.I2C_ADDRESS, 1)
        return t[0]


    def _write_reg_data(self, reg, data):
        """ Escribe el byte 'data' en un registro del sensor """
        # mirar comentarios de _read_word
        self.i2c.writeto(APDS9930.I2C_ADDRESS, bytearray((reg|0x80, data)))


    def activar_proximidad(self):
        """ Activa el sensor de proximidad """
        # Modificamos los bits necesarios del Enable Register segun indica el datasheet
        Enable_Register = 0x00
        en_reg_data = self._read_byte(Enable_Register)
        # print("antes {0:08b}".format(en_reg_data))  # podemos mostrar como estaba el registro antes de modificarlo
        en_reg_data_enmascarado = en_reg_data | 0b00000100  # con esta mascara (0x04), ponemos el bit Proximity Enable a 1 sin modificar los demas
        # si quisiesemos poner ese bit a 0 sin modificar los demas, utilizariamos esta mascara:
        # en_reg_data = en_reg_data & 0b11111011
        self._write_reg_data(Enable_Register, en_reg_data_enmascarado)
        # print("despues {0:08b}".format(en_reg_data))  # podemos mostrar como estaba el registro despues de modificarlo


    def get_proximidad(self):
        """ Devuelve lectura del sensor de proximidad """
        # lee el dato de 16 bits del registro PDATA
        # NOTA este valor habria que dividirlo por la ganancia, no lo vamos a hacer por simplicidad
        return self._read_word(0x18)  # 0x18 PDATAL, 0x19 PDATAH


## Juego con sensor o sin él

### main.py

In [ ]:
import network
import utime as time

from credenciales import ssid, password
from apds9930 import APDS9930
from juego import Juego

def conectar_wifi():
    print('\nConectandose a la wifi...', end='')
    red = network.WLAN(network.STA_IF)
    red.active(True)
    red.connect(ssid, password)
    while not red.isconnected():
        time.sleep(0.1)
    print('conectado!')
    print(red.ifconfig())


i2c=I2C(sda=Pin(4), scl=Pin(5))
try:
    sensor = APDS9930(i2c)
    sensor.activar_proximidad()
except Exception as e:
    sensor = None

conectar_wifi()
juego = Juego("Rober .H")
juego.metodo_entrada(sensor)
juego.start()

### juego.py

In [ ]:
from machine import Pin, I2C
import urandom as random
import utime as time

from credenciales import ssid, password
from mimqtt import Mimqtt


class Juego:
    def __init__(self, jugador):
        self.led = Pin(2, Pin.OUT)
        self.boton = Pin(0, Pin.IN)
        self.led.value(1)
        self.jugador = jugador
        self.mimqtt = Mimqtt()
        # inicializar el sensor en el constructor de Juego
        self.sensor = None

    def metodo_entrada(self, mode):
        self.sensor = mode

    def start(self):
        print()
        if self.sensor:
            print('Activa el sensor cuando se encienda el led')
        else:
            print("Pulsa el botón cuando se encienda el led")
        while True:
            self.encender_led_aleatorio()
            inicio = time.ticks_ms()
            self.apagar_led_boton()
            final = time.ticks_ms()
            tiempo = time.ticks_diff(final, inicio)
            if tiempo > 5000:
                print('Has tardado más de 5 segundos')
                print("\n")
                print('Reiniciando ...')
                self.start()
            elif tiempo < 10:
                if self.sensor:
                    print('El sensor estaba activado antes de encenderse el led')
                else:
                    print("El botón estaba activado antes de encenderse el led")
            else:
                print('Has apagado el botón en {} ms'.format(tiempo))
                if self.sensor:
                    metodo = "sensor"
                else:
                    metodo = "boton"
                self.mimqtt.enviar_mqtt(self.jugador, metodo, tiempo)

    def encender_led_aleatorio(self):
        aleatorio = random.getrandbits(12)
        aleatorio += 3000
        time.sleep_ms(aleatorio)
        self.led.value(0)

    def apagar_led_boton(self):
        if self.sensor:
            while not self.led.value():
                proximidad = self.sensor.get_proximidad()
                if proximidad is not 0:
                    print("Sensor activado a {}!".format(proximidad))
                    self.led.value(1)
        else:
            while self.boton.value() == 1:
                time.sleep_ms(1)
            self.led.value(1)

# `btree`: una base de datos para iot

In [ ]:
import btree

try:
    f = open('mydb', "r+b") # lectura con permisos de escritura pero que escriba desde el principio secuencia de bytes
except OSError:
    f = open('mydb', "w+b")

db = btree.open(f) # instanciamos la base de datos y la abrimos

db[b'3'] = b'tres'
db[b'1'] = b'uno'
db[b'2'] = b'dos' # estamos guardando en memoria

db.flush() # grabamos los datos en la bd

for word in db.values(b'2'): # punto de partida el 2
    print(word)

del db[b'2'] # borramos

# si no hacemos flush, no se guardan al cerrar

db.close() # cerramos base de datos

f.close() # cerramos el fichero que abrimos

## 